In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
county_geo_gdf = gpd.read_file('./dataset/cb_2017_us_county_5m/cb_2017_us_county_5m.shp')

We want to find the exact county of an airport so we apply a geometry dataset and find if the airport is in specific county

In [4]:
county_geo_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,04,015,00025445,0500000US04015,04015,Mohave,06,34475503964,387344307,"POLYGON ((-114.75562 36.08717, -114.75364 36.0..."
1,22,105,00559500,0500000US22105,22105,Tangipahoa,15,2049488093,136678798,"POLYGON ((-90.56717 30.82481, -90.56719 30.999..."
2,16,063,00395624,0500000US16063,16063,Lincoln,06,3111451190,11606076,"POLYGON ((-114.59461 43.19834, -114.37496 43.1..."
3,27,119,00659505,0500000US27119,27119,Polk,06,5105067510,69169913,"POLYGON ((-97.14667 48.17148, -97.14570 48.173..."
4,38,017,01034226,0500000US38017,38017,Cass,06,4571107601,7732062,"POLYGON ((-97.70603 47.23998, -97.45151 47.239..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,101,00835872,0500000US31101,31101,Keith,06,2749531887,124672175,"POLYGON ((-102.05554 41.22155, -102.05526 41.3..."
3229,28,001,00695726,0500000US28001,28001,Adams,06,1197464269,65273640,"POLYGON ((-91.65501 31.25181, -91.65403 31.255..."
3230,36,069,00974133,0500000US36069,36069,Ontario,06,1668114449,47820988,"POLYGON ((-77.61167 42.76317, -77.58822 42.762..."
3231,54,053,01560254,0500000US54053,54053,Mason,06,1115633278,36174540,"POLYGON ((-82.22157 38.78719, -82.21727 38.795..."


In [26]:
county_geo_gdf[county_geo_gdf['NAME'] == 'Dane']
county_geo_gdf = county_geo_gdf[['STATEFP',	'COUNTYFP',	'NAME' , 'geometry']]
county_geo_gdf

,STATEFP,COUNTYFP,NAME,geometry
0,04,015,Mohave,"POLYGON ((-114.75562 36.08717, -114.75364 36.0..."
1,22,105,Tangipahoa,"POLYGON ((-90.56717 30.82481, -90.56719 30.999..."
2,16,063,Lincoln,"POLYGON ((-114.59461 43.19834, -114.37496 43.1..."
3,27,119,Polk,"POLYGON ((-97.14667 48.17148, -97.14570 48.173..."
4,38,017,Cass,"POLYGON ((-97.70603 47.23998, -97.45151 47.239..."
...,...,...,...,...
3228,31,101,Keith,"POLYGON ((-102.05554 41.22155, -102.05526 41.3..."
3229,28,001,Adams,"POLYGON ((-91.65501 31.25181, -91.65403 31.255..."
3230,36,069,Ontario,"POLYGON ((-77.61167 42.76317, -77.58822 42.762..."
3231,54,053,Mason,"POLYGON ((-82.22157 38.78719, -82.21727 38.795..."


In [6]:
from shapely.geometry import Point

In [27]:
airline_df = pd.read_csv('./dataset/geo_airports_week.csv')
airline_df.head()

,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,IATA,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
1,10140,2018,2,1,269,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
2,10140,2018,3,5,270,3,0,0,2,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
3,10140,2018,4,0,270,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
4,10140,2018,5,0,266,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194


We have an assumption that people would take the flights at the airports within 20 miles from their county. So we create the geometry circle to find if the circle intersect any other counties within 20 miles.

In [28]:
points  = [Point(x, y) for x, y in zip(airline_df['LONGITUDE'], airline_df['LATITUDE'])]
# Convert the radius from miles to degrees
# 1 degree = 69.172 miles
radius = 20 / 69.172 
circles = [Point(p.x, p.y).buffer(radius) for p in points]
airline_gdf = gpd.GeoDataFrame(airline_df, geometry=circles)
airline_gdf.crs = {'init': 'epsg:4326'}
airline_gdf

C:\Users\bianh\AppData\Roaming\Python\Python39\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,IATA,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,geometry
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0..."
1,10140,2018,2,1,269,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0..."
2,10140,2018,3,5,270,3,0,0,2,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0..."
3,10140,2018,4,0,270,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0..."
4,10140,2018,5,0,266,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21177,15624,2022,27,2,216,0,0,0,2,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",VPS,Eglin Air Force Base,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454..."
21178,15624,2022,28,5,210,1,0,0,4,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",VPS,Eglin Air Force Base,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454..."
21179,15624,2022,29,5,209,2,0,0,3,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",VPS,Eglin Air Force Base,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454..."
21180,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",VPS,Eglin Air Force Base,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454..."


In [30]:
# we have the matching county by intersect
matching_counties = []
airline_county_gdf = gpd.sjoin(airline_gdf, county_geo_gdf, how='left', op='intersects')

d:\App\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\bianh\AppData\Local\Temp\ipykernel_22256\3452997435.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4269

  airline_county_gdf = gpd.sjoin(airline_gdf, county_geo_gdf, how='left', op='intersects')


In [31]:
airline_county_gdf

,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,...,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,geometry,index_right,STATEFP,COUNTYFP,NAME
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",971,35,061,Valencia
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",2289,35,001,Bernalillo
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",1746,35,043,Sandoval
1,10140,2018,2,1,269,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,Albuquerque,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21180,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",1916,12,113,Santa Rosa
21180,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa
21181,15624,2022,52,2,22,1,0,0,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",574,12,131,Walton
21181,15624,2022,52,2,22,1,0,0,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,Valparaiso,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",1916,12,113,Santa Rosa


In [33]:
merged_gdf = airline_county_gdf.merge(county_geo_gdf, on = ['STATEFP',	'COUNTYFP',	'NAME'])
merged_gdf

,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,...,STATE,COUNTRY,LATITUDE,LONGITUDE,geometry_x,index_right,STATEFP,COUNTYFP,NAME,geometry_y
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
1,10140,2018,2,1,269,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
2,10140,2018,3,5,270,3,0,0,2,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
3,10140,2018,4,0,270,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
4,10140,2018,5,0,266,0,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109713,15624,2022,27,2,216,0,0,0,2,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa,"POLYGON ((-86.79243 30.68514, -86.78569 30.996..."
109714,15624,2022,28,5,210,1,0,0,4,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa,"POLYGON ((-86.79243 30.68514, -86.78569 30.996..."
109715,15624,2022,29,5,209,2,0,0,3,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa,"POLYGON ((-86.79243 30.68514, -86.78569 30.996..."
109716,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa,"POLYGON ((-86.79243 30.68514, -86.78569 30.996..."


In [38]:
airline_county_gdf['COUNTY_SHAPE'] = merged_gdf['geometry_y']
airline_county_gdf

,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,...,STATE,COUNTRY,LATITUDE,LONGITUDE,geometry,index_right,STATEFP,COUNTYFP,NAME,COUNTY_SHAPE
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",971,35,061,Valencia,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",2289,35,001,Bernalillo,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
0,10140,2018,1,1,362,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",1746,35,043,Sandoval,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
1,10140,2018,2,1,269,1,0,0,0,"Albuquerque, NM: Albuquerque International Sun...",...,NM,USA,35.040222,-106.609194,"POLYGON ((-106.32006 35.04022, -106.32145 35.0...",591,35,057,Torrance,"POLYGON ((-106.46487 34.60838, -106.46671 34.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21180,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",1916,12,113,Santa Rosa,"POLYGON ((-80.78417 35.50602, -80.73738 35.505..."
21180,15624,2022,30,3,206,1,0,1,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",52,12,091,Okaloosa,"POLYGON ((-80.78417 35.50602, -80.73738 35.505..."
21181,15624,2022,52,2,22,1,0,0,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",574,12,131,Walton,"POLYGON ((-80.78417 35.50602, -80.73738 35.505..."
21181,15624,2022,52,2,22,1,0,0,1,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,FL,USA,30.483250,-86.525400,"POLYGON ((-86.23627 30.48325, -86.23766 30.454...",1916,12,113,Santa Rosa,"POLYGON ((-80.78417 35.50602, -80.73738 35.505..."


In [39]:
airline_county_gdf.to_csv('./dataset/airline_county_gdf.csv', index = False)

In [46]:
covid_df = pd.read_csv('./dataset/covid_county_week.csv', dtype={'STATEFP': str, 'COUNTYFP': str})
covid_df

,fips,YEAR,WEEK,cases,deaths,county,state,stay_at_home_announced,stay_at_home_effective,total_population,...,rain,snow,thunder,visibility,max_wind_speed,lat,lon,weekly_cases,STATEFP,COUNTYFP
0,1001,2020,13,6,0.0,Autauga,AL,no,no,55049.0,...,1.0,0.0,0.0,9.633333,10.333333,32.534928,-86.642748,6.0,01,001
1,1001,2020,14,12,0.0,Autauga,AL,no,no,55049.0,...,2.0,0.0,1.0,9.971429,8.328571,32.534928,-86.642748,6.0,01,001
2,1001,2020,15,19,1.0,Autauga,AL,yes,yes,55049.0,...,5.0,0.0,2.0,9.128571,11.842857,32.534928,-86.642748,7.0,01,001
3,1001,2020,16,27,2.0,Autauga,AL,yes,yes,55049.0,...,3.0,0.0,2.0,9.785714,13.414286,32.534928,-86.642748,8.0,01,001
4,1001,2020,17,37,2.0,Autauga,AL,yes,yes,55049.0,...,3.0,0.0,2.0,9.200000,12.128571,32.534928,-86.642748,10.0,01,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112522,56045,2020,45,306,0.0,Weston,WY,no,no,7175.0,...,0.0,0.0,0.0,NaN,NaN,43.840417,-104.567663,142.0,56,045
112523,56045,2020,46,339,0.0,Weston,WY,no,no,7175.0,...,0.0,0.0,0.0,NaN,NaN,43.840417,-104.567663,33.0,56,045
112524,56045,2020,47,372,0.0,Weston,WY,no,no,7175.0,...,0.0,0.0,0.0,NaN,NaN,43.840417,-104.567663,33.0,56,045
112525,56045,2020,48,396,1.0,Weston,WY,no,no,7175.0,...,0.0,0.0,0.0,NaN,NaN,43.840417,-104.567663,24.0,56,045


We create a geometry shape for each county

In [52]:
# geometry = [Point(x, y) for x, y in zip(covid_df['lon'], covid_df['lat'])]
# covid_gdf = gpd.GeoDataFrame(covid_df, geometry=geometry)
# covid_gdf
covid_gdf = covid_df.merge(county_geo_gdf, on=['STATEFP',	'COUNTYFP'])

In [53]:
covid_gdf

,fips,YEAR,WEEK,cases,deaths,county,state,stay_at_home_announced,stay_at_home_effective,total_population,...,thunder,visibility,max_wind_speed,lat,lon,weekly_cases,STATEFP,COUNTYFP,NAME,geometry
0,1001,2020,13,6,0.0,Autauga,AL,no,no,55049.0,...,0.0,9.633333,10.333333,32.534928,-86.642748,6.0,01,001,Autauga,"POLYGON ((-86.91759 32.66417, -86.81657 32.660..."
1,1001,2020,14,12,0.0,Autauga,AL,no,no,55049.0,...,1.0,9.971429,8.328571,32.534928,-86.642748,6.0,01,001,Autauga,"POLYGON ((-86.91759 32.66417, -86.81657 32.660..."
2,1001,2020,15,19,1.0,Autauga,AL,yes,yes,55049.0,...,2.0,9.128571,11.842857,32.534928,-86.642748,7.0,01,001,Autauga,"POLYGON ((-86.91759 32.66417, -86.81657 32.660..."
3,1001,2020,16,27,2.0,Autauga,AL,yes,yes,55049.0,...,2.0,9.785714,13.414286,32.534928,-86.642748,8.0,01,001,Autauga,"POLYGON ((-86.91759 32.66417, -86.81657 32.660..."
4,1001,2020,17,37,2.0,Autauga,AL,yes,yes,55049.0,...,2.0,9.200000,12.128571,32.534928,-86.642748,10.0,01,001,Autauga,"POLYGON ((-86.91759 32.66417, -86.81657 32.660..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112522,56045,2020,45,306,0.0,Weston,WY,no,no,7175.0,...,0.0,NaN,NaN,43.840417,-104.567663,142.0,56,045,Weston,"POLYGON ((-105.08078 43.96622, -105.07928 44.1..."
112523,56045,2020,46,339,0.0,Weston,WY,no,no,7175.0,...,0.0,NaN,NaN,43.840417,-104.567663,33.0,56,045,Weston,"POLYGON ((-105.08078 43.96622, -105.07928 44.1..."
112524,56045,2020,47,372,0.0,Weston,WY,no,no,7175.0,...,0.0,NaN,NaN,43.840417,-104.567663,33.0,56,045,Weston,"POLYGON ((-105.08078 43.96622, -105.07928 44.1..."
112525,56045,2020,48,396,1.0,Weston,WY,no,no,7175.0,...,0.0,NaN,NaN,43.840417,-104.567663,24.0,56,045,Weston,"POLYGON ((-105.08078 43.96622, -105.07928 44.1..."


In [54]:
covid_gdf.to_csv('./dataset/covid_geo.csv', index=False)

In [49]:
covid_airline_gdf = airline_county_gdf.merge(covid_df, on = ['STATEFP',	'COUNTYFP', 'YEAR', 'WEEK'])

In [50]:
covid_airline_gdf

,ORIGIN_AIRPORT_ID,YEAR,WEEK,CANCELLED,COUNT_FLIGHTS,WEATHER_CANCELLED,NAS_CANCELLED,SEC_CANCELLED,CARRIER_CANCELLED,Description,...,percent_below_poverty,fog,rain,snow,thunder,visibility,max_wind_speed,lat,lon,weekly_cases
0,10140,2020,11,7,318,1,2,2,2,"Albuquerque, NM: Albuquerque International Sun...",...,18.7,0.0,3.0,0.0,1.0,9.900000,16.820000,35.051368,-106.670163,10.0
1,10140,2020,11,7,318,1,2,2,2,"Albuquerque, NM: Albuquerque International Sun...",...,14.4,0.0,0.0,0.0,0.0,NaN,NaN,35.688552,-106.866074,2.0
2,10140,2020,12,58,331,4,7,44,3,"Albuquerque, NM: Albuquerque International Sun...",...,18.7,0.0,3.0,0.0,0.0,9.957143,18.900000,35.051368,-106.670163,24.0
3,10140,2020,12,58,331,4,7,44,3,"Albuquerque, NM: Albuquerque International Sun...",...,14.4,0.0,0.0,0.0,0.0,NaN,NaN,35.688552,-106.866074,5.0
4,10140,2020,13,134,330,0,0,127,7,"Albuquerque, NM: Albuquerque International Sun...",...,23.2,0.0,0.0,0.0,0.0,10.000000,25.400000,34.715506,-106.808997,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16297,15624,2020,47,0,64,0,0,0,0,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,12.0,0.0,0.0,0.0,0.0,9.885714,9.700000,30.691297,-86.591760,708.0
16298,15624,2020,48,3,75,0,0,0,3,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,17.4,3.0,5.0,0.0,1.0,8.771429,12.028571,30.643586,-86.169696,177.0
16299,15624,2020,48,3,75,0,0,0,3,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,12.0,3.0,5.0,0.0,1.0,8.771429,12.028571,30.691297,-86.591760,542.0
16300,15624,2020,49,1,48,0,0,1,0,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",...,17.4,0.0,0.0,0.0,0.0,10.000000,15.900000,30.643586,-86.169696,172.0


In [51]:
covid_airline_gdf.to_csv('./dataset/covid_airline.csv', index = False)